# Google Earth Engine (GEE) with Python

In the following example we are working with the Google Earth Engine (GEE) using the Python API. 
The goal of this example is to familiarize you with some of the data that can be found on the GEE and how to handle it.
Some parts of the code used will be relativly complicated to understand. This is a concequence of how the data is structured in the GEE. For now it is not important to understand each line of code. The goal here is make sure you understand the general idea behind the operations and get an impression of what is possible with the GEE in combination with the Python API. For examples and tutorials please visit the [examples](https://github.com/giswqs/geemap/tree/master/examples) page

Make sure to install the following packages in your environment before working with this notebook:
- geemap

In [3]:
!pip install geemap # ee is installed automatically, but geemap is not. 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 7.1 MB/s 
     |████████████████████████████████| 3.5 MB 40.7 MB/s 
     |████████████████████████████████| 99 kB 2.8 MB/s 
     |████████████████████████████████| 98 kB 3.7 MB/s 
     |████████████████████████████████| 1.2 MB 31.1 MB/s 
     |████████████████████████████████| 46 kB 2.1 MB/s 
     |████████████████████████████████| 130 kB 44.5 MB/s 
     |████████████████████████████████| 3.7 MB 42.3 MB/s 
     |████████████████████████████████| 1.3 MB 36.8 MB/s 
     |████████████████████████████████| 198 kB 44.1 MB/s 
     |████████████████████████████████| 55 kB 1.9 MB/s 
     |████████████████████████████████| 128 kB 46.3 MB/s 
     |████████████████████████████████| 1.6 MB 35.9 MB/s 
     |████████████████████████████████| 79 kB 5.0 MB/s 
  Created wheel for ee-extra: filename=ee_extra-0.0.14-py3-none-any.whl size=209116 sha256=8568bc598

In [4]:
import geemap
import ee
import os

In [45]:
#Mounting the drive 
from google.colab import drive 
drive.mount ("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=S3DP-s__EZMHPDmBoYatzNEYe_0Xj2tDwtSOzR9-ko0&tc=aNZ_IzikEXMGXOBSQSgxy06FLcB7TmRE2-uDmlry0NI&cc=MJRKGYn2gjtvjxxEJbOqMz3azkwhcgFelxb0C_BexE0

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfgeXvtjbV9fhqqSS3l7-uUhqCn6HMwR0zR0GD8JImb16mj551pYkJT_K-s

Successfully saved authorization token.


In [32]:
# Import elevation data. We use the SRTM Digital Elevation Data Version 4
dataset = ee.Image('CGIAR/SRTM90_V4')
elevation = dataset.select('elevation')
# Import countries feature data
Boundaries = ee.FeatureCollection('FAO/GAUL/2015/level0')


In [38]:
# Choose country
country_name = "Switzerland"
country_filter = Boundaries.filter(ee.Filter.eq('ADM0_NAME', country_name))

In [39]:
# Clipping image to feature
elevation_country = elevation.clip(country_filter)


In [40]:
# Choose visualization features. Here minimum, maximum, and color choice

vis_slope = {
    'min': 0,
    'max': 90,
    'palette': ['3D59AB','FF9912','3D9140','EE1289','FCE6C9']
}

In [41]:
# Calculate slope and reproject to 1000m resolution
slopeScaled = ee.Terrain.slope(elevation_country.reproject(crs = 'EPSG:4326', scale = 1000))

In [43]:
# set our initial map parameters for the country
zoomlevel=4.5

# This will generate the basic map, but not visualize it
Map = geemap.Map(zoom=zoomlevel)


# With addLayer we can add datasets to the map
Map.addLayer(slopeScaled, vis_slope)

Map.centerObject(country_filter.geometry())

# To view the map simply type Map 
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [47]:
#to save the calculated slope in Google Drive as GeoTiff(.tif)
task = ee.batch.Export.image.toDrive(
                                     image = slopeScaled,
                                     description = 'Slope',
                                     fileFormat = 'GeoTIFF',
                                     folder = "ASA",
                                     region = country_filter.geometry(),
                                     scale = 1000,   #in meters
)
task.start()
